# 크롤링

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import urllib.request 
from selenium.webdriver import Chrome
import re     
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt
import pandas as pd

In [55]:
def get_videos (channel): 
    
    # 브라우저 열기
    delay=3
    browser = Chrome()
    browser.implicitly_wait(delay)

    # url을 받아 창을 열고, 최대화
    start_url  = channel
    browser.get(start_url)  
    browser.maximize_window()

    # 영상목록 클릭 후 body 찾기
    body = browser.find_element_by_css_selector('body')

    
    # 5번 밑으로 내리기 -> 타임라인 업데이트를 위해서!
    num_of_pagedowns = 2
    while num_of_pagedowns:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        num_of_pagedowns -= 1
    
    # 파싱 후 크롤링
    html0 = browser.page_source
    soup = BeautifulSoup(html0,'lxml')
    
    videos = soup.select('h3 > a')

    title = []
    url = []

    for video in videos:
        if video.get('href')[:7] != '/watch?':
            pass
        else:
            title.append(video.text)
            url.append('https://www.youtube.com/' + video.get('href'))
    
   

    browser.close() 
    return url, title


'https://www.youtube.com/channel/UC3HH4dZj3CZmrCC0MuDliww'

'https://www.youtube.com/channel/UC3HH4dZj3CZmrCC0MuDliww'

In [57]:
url_list, title_list = get_videos('https://www.youtube.com/channel/UC3HH4dZj3CZmrCC0MuDliww')

In [62]:

for i in reversed(range(len(url_list))):
    name = title_list[i]
    start_url = url_list[i]
    browser = Chrome()
    browser.get(start_url)

    time.sleep(2)

    html0 = browser.page_source
    html = BeautifulSoup(html0,'html.parser')

    start_date = html.find('div',{'id':'info-strings','class':'style-scope ytd-video-primary-info-renderer'}).text
    start_date = re.sub(r'.', '', start_date, count = 1)
    print(start_date + " " + name)
    view = html.find('span',{'class':'view-count style-scope ytd-video-view-count-renderer'}).text
    view = re.sub(r'.', '', view, count = 4)
    print(view)
    time.sleep(1)
    #좋아요수
    likes_num = html.find('yt-formatted-string',{'id':'text',
                                                 'class':'style-scope ytd-toggle-button-renderer style-text',
                                                 'aria-label':re.compile('좋아요')}).text+'개'
        
        
    #싫어요수
    unlikes_num = html.find('yt-formatted-string',{'id':'text',
                                                   'class':'style-scope ytd-toggle-button-renderer style-text',
                                                   'aria-label':re.compile('싫어요')}).text+'개'
    print(likes_num,unlikes_num)
    insert_data = pd.DataFrame({
    '날짜':[start_date],
    '제목':title_list,
    '유튜브 조회수':[view],
    'Video_URL':[url],

    })
    df = df.append(insert_data)

browser.quit()
df

019. 9. 20. 전설은 영원하다 (결말포함/다크나이트라이즈)
1,871,213회
9.3천개 547개


NameError: name 'url' is not defined

In [51]:
def info_crawler (url_list,html):
    info_df = pd.DataFrame({'name':[],
                     'thumbnail':[],
                     'view':[],
                     'previous_time':[],
                     'video_url':[],
                     'start_date':[],
                     'comment':[],
                     'likes_num':[],
                     'unlikes_num':[]})
    
    # 리스트 준비
    titles = html.find_all('a',{'id':'video-title'})
    thums = html.find_all('a',{'id':'thumbnail'})
    metas =html.find_all('div',{'id':'metadata-line'})
    browser = Chrome()

    # 제목, 썸네일, url, 조회수, 게시일 추출
    for i in range(0,3): # 시간제약상 5개만
        name = titles[i].text
        thum = thums[i].find('img')['src']
        url = url_list[i]
        meta0 = metas[i]
    
        view = meta0.find_all('span',
            {'class':'style-scope ytd-grid-video-renderer'})[0].text.split()[1]
        previous = meta0.find_all('span',
                    {'class':'style-scope ytd-grid-video-renderer'})[1].text
    
    
        start_url = url_list[i]
    
        browser = Chrome()
        browser.implicitly_wait(3)
        browser.get(start_url)
    
        body = browser.find_element_by_tag_name('body')
    
        time.sleep(3)

        # 영상별 댓글수, 좋아요 및 싫어요 개수 추출
        num_of_pagedowns = 2
        while num_of_pagedowns:
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(3) 
            num_of_pagedowns -= 1
        html0 = browser.page_source
        html = BeautifulSoup(html0,'html.parser')
    
        browser.close()
        
        start_date = html.find('span',
            {'class':'date style-scope ytd-video-secondary-info-renderer'}).text
        
        try:
            comment = html.find('h2',{'id':'count'}).find('yt-formatted-string').text
        except:
            pass
    
        #좋아요수
        likes_num = html.find('yt-formatted-string',{'id':'text',
                'class':'style-scope ytd-toggle-button-renderer style-text',
                                'aria-label':re.compile('좋아요')}).text+'개'
        
        
        #싫어요수
        unlikes_num = html.find('yt-formatted-string',{'id':'text',
                    'class':'style-scope ytd-toggle-button-renderer style-text',
                                    'aria-label':re.compile('싫어요')}).text+'개'
    
        insert_data = pd.DataFrame({'name':[name],
                     'thumbnail':[thum],
                     'view':[view],
                     'previous_time':[previous],
                     'video_url':[url],
                     'start_date':[start_date],
                     'comment':[comment],
                     'likes_num':[likes_num],
                     'unlikes_num':[unlikes_num]})
    
        info_df = info_df.append(insert_data)
    info_df.index = range(len(info_df))
    return info_df

In [52]:
v1 = info_crawler(ch1,ll)
v1

AttributeError: 'list' object has no attribute 'find_all'

In [37]:
def get_video(channel):
    feature = input('검색어를 입력하시오 : ')
    
    driver = Chrome()
    driver.get(channel)
    
    n = 3
    while n > 0:
        print('웹페이지를 불러오는 중입니다..' + '..' * n)
        time.sleep(1)
        n -= 1
    '''
    src = driver.find_element_by_xpath('//*[@id="search"]')
    src.send_keys(feature)
    src.send_keys(Keys.RETURN)
    
    n = 2
    while n > 0:
        print('검색 결과를 불러오는 중입니다..' + '..' * n)
        time.sleep(1)
        n -= 1
        
    print('데이터 수집 중입니다....')
    '''
               
    html = driver.page_source
    soup = BeautifulSoup(html)
    
    df_title = []
    df_link = []
    df_writer = []
    df_view = []
    df_date = []
    
    for i in range(len(soup.find_all('ytd-video-meta-block', 'style-scope ytd-video-renderer byline-separated'))):
        title = soup.find_all('a', {'id' : 'video-title'})[i].text.replace('\n', '')
        link = 'https://www.youtube.com/' + soup.find_all('a', {'id' : 'video-title'})[i]['href']
        writer = soup.find_all('ytd-channel-name', 'long-byline style-scope ytd-video-renderer')[i].text.replace('\n', '').split(' ')[0]
        view = soup.find_all('ytd-video-meta-block', 'style-scope ytd-video-renderer byline-separated')[i].text.split('•')[1].split('\n')[3]
        date = soup.find_all('ytd-video-meta-block', 'style-scope ytd-video-renderer byline-separated')[i].text.split('•')[1].split('\n')[4]
    
        df_title.append(title)
        df_link.append(link)
        df_writer.append(writer)
        df_view.append(view)
        df_date.append(date)
          
    df_just_video = pd.DataFrame(columns=['영상제목', '채널명', '영상url', '조회수', '영상등록날짜'])

    df_just_video['영상제목'] = df_title
    df_just_video['채널명'] = df_writer
    df_just_video['영상url'] = df_link
    df_just_video['조회수'] = df_view
    df_just_video['영상등록날짜'] = df_date
          
    df_just_video.to_csv('df_just_video.csv', encoding='utf-8-sig', index=False)
    
    driver.close()
          
    result = input('데이터프레임 저장이 완료되었습니다! 데이터프레임을 조회하시겠습니까? (y/n)')
    if result == 'y':
        display(df_just_video)
        question = input('원하는 영상을 재생하시겠습니까? (y/n)')
        if question == 'y':
            button = int(input('재생하고자 하는 영상의 번호(출력된 표 가장 왼쪽의 번호)를 입력해주세요.'))
            driver = webdriver.Chrome(path)
            driver.get(df_just_video['영상url'][button])
        else:
            return '프로그램을 종료합니다.'
    else:
        return '프로그램을 종료합니다.'

In [38]:
get_video('https://www.youtube.com/channel/UC3HH4dZj3CZmrCC0MuDliww')

검색어를 입력하시오 : 
웹페이지를 불러오는 중입니다........
웹페이지를 불러오는 중입니다......
웹페이지를 불러오는 중입니다....
데이터프레임 저장이 완료되었습니다! 데이터프레임을 조회하시겠습니까? (y/n)y


,영상제목,채널명,영상url,조회수,영상등록날짜


원하는 영상을 재생하시겠습니까? (y/n)n


'프로그램을 종료합니다.'